# Pandas DataFrame

In [ ]:
import pandas as pd  # python community standard 

filename = "../data/raw/原始資料.xlsx"

xls = pd.ExcelFile(filename, engine='openpyxl')
df = pd.read_excel(xls, index_col=0)

In [ ]:
df = df[['代號', '數量']]

In [ ]:
# show first 20 rows

df.head(20)

In [ ]:
# Select data with whose <代號 A> in the dataframe

df_A = df[df['代號']=='A']

In [ ]:
df_A.tail(10)

In [ ]:
# Sum 數量 for each 代號 each 交易日期
# Make 交易日期 from index to column

df.reset_index(inplace=True)  # or df = df.reset_index()

In [ ]:
df_sum = df.groupby(['代號', '交易日期'], as_index=False)['數量'].sum()

In [ ]:
df_sum.head(10)

In [ ]:
df_sum.tail(10)

In [ ]:
df_sum[df_sum['代號']=='A']

In [ ]:
# Identify unique 代號
import numpy as np

unique_product_code = np.unique(df_sum['代號'])

In [ ]:
unique_product_code

In [ ]:
product_df_list = []  # python list

for code in unique_product_code:
    product_df_list.append(df_sum[df_sum['代號']==code])

In [ ]:
# Let us see what we have in the list

product_df_list[0]

In [ ]:
product_df_list[-1]

In [ ]:
product_df_list = []

for code in unique_product_code:
    product_df_list.append(df_sum[df_sum['代號']==code].set_index("交易日期"))

In [ ]:
product_df_list[0]

In [ ]:
# Replace all the 數量 with the 代號

product_df_list = []

for code in unique_product_code:
    temp = df_sum[df_sum['代號']==code].set_index("交易日期")
    temp.drop(labels=['代號'], inplace=True, axis=1)
    temp.rename(columns={'數量': code}, inplace=True)
    product_df_list.append(temp)

In [ ]:
product_df_list[0]

In [ ]:
df_parsed = pd.concat(product_df_list, axis=1)

In [ ]:
df_parsed['A'].head(20)

In [ ]:
df_parsed['A'].fillna(0).plot(figsize=(8,8))

## Timestamp

In [ ]:
df_sum['交易日期'].iloc[0]

In [ ]:
timestamp_begin = df_sum['交易日期'].iloc[0]

In [ ]:
# Here we show how to add days on the timestamp

from datetime import timedelta
    
timestamp_plus_one = timestamp_begin + timedelta(days=1)

In [ ]:
timestamp_plus_one

In [ ]:
# Interpolate timestamp in the dataframe index

df_parsed['A'].resample('1D').sum()

In [ ]:
df_parsed['A']

We can see that 42 days are added to the pandas series

# Time series model - Let us start :)

Let us focus on 代號 A...

Using 90 days data to predict the one week sum after one month.

For example: using data from 01.01.2018 - 03.31.2018 to predict sum during 05.01.2018 - 05.07.2018

Now we have all the tool we need, let us construct the data for machine learning.

In [ ]:
df_A = df_parsed[['A']]
df_A = df_A.resample('1D').sum()
df_A.fillna(0, inplace=True)  # replace all the NaN with 0

In [ ]:
# Train test split
train_size = 0.8

df_train = df_A[:int(train_size * len(df_A))]
df_test = df_A[int(train_size * len(df_A)):]

In [ ]:
df_train.tail(5)

In [ ]:
df_test.head(5)

In [ ]:
from sklearn.preprocessing import MinMaxScaler

normalizer = MinMaxScaler()

df_train_normalized = normalizer.fit_transform(df_train)
df_test_normalized = normalizer.transform(df_test)

In [ ]:
history_points = 90
future = 30
duration = 7

X_train = np.array([df_train_normalized[i : i + history_points] for i in range(len(df_train_normalized) - history_points)])
y_train = np.array([df_train_normalized[i + history_points + future: i + history_points + future + duration].sum() for i in range(len(df_train_normalized) - history_points - future)])

In [ ]:
y_train = np.array([df_train_normalized[i + history_points + future: i + history_points + future + duration].sum() for i in range(len(df_train_normalized) - history_points - future)])

In [ ]:
X_train.shape

In [ ]:
y_train.shape

In [ ]:
import matplotlib.pyplot as plt

plt.plot(y_train)

In [ ]:
plt.plot(y_test)

In [ ]:
X_train = X_train[:len(y_train)]

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, LSTM, Input, Activation, concatenate
from tensorflow.keras import optimizers

# build LSTM model

def lstm_model():
    tf.random.set_seed(20)
    np.random.seed(10)
    lstm_input = Input(shape=(history_points, 1), name='lstm_input')

    inputs = LSTM(30, name='first_layer')(lstm_input)
    inputs = Dense(10)(inputs)
    inputs = Dense(1)(inputs)
    output = Activation('linear', name='output')(inputs)

    model = Model(inputs=lstm_input, outputs=output)
    
    return model

In [ ]:
X_test = np.array([df_test_normalized[i : i + history_points] for i in range(len(df_test_normalized) - history_points)])
y_test = np.array([df_test_normalized[i + history_points + future: i + history_points + future + duration].sum() for i in range(len(df_test_normalized) - history_points - future)])
X_test = X_test[:len(y_test)]

In [ ]:
model = lstm_model()

adam = optimizers.Adam(lr = 0.001)
model.compile(optimizer=adam, loss='mse')
model.fit(x=X_train, y=y_train, batch_size=4, epochs=5, shuffle=True, validation_data=(X_test, y_test))

In [ ]:
y_pred = model.predict(X_test)

fig, ax = plt.subplots(figsize=(8,6))

ax.plot(y_pred, label='LSTM prediction')
ax.plot(y_test, label='Ground_truth')

ax.set_title("Validation set")

ax.legend()

In [ ]:
y_pred = model.predict(X_train)

fig, ax = plt.subplots(figsize=(16,12))

ax.plot(y_pred, label='LSTM prediction')
ax.plot(y_train, label='Ground_truth')

ax.set_title("Training set")

ax.legend()

The model captures something but not response fast enough to the short time movement. Let us see if we can improve the model.

In [ ]:
df_A.diff()

In [ ]:
df_A

In [ ]:
df_A['A_diff'] = df_A.diff()

In [ ]:
df_A.head(5)

In [ ]:
df_train = df_A[:int(train_size * len(df_A))].fillna(0)
df_test = df_A[int(train_size * len(df_A)):].fillna(0)

normalizer = MinMaxScaler()

df_train_normalized = normalizer.fit_transform(df_train)
df_test_normalized = normalizer.transform(df_test)

history_points = 90
future = 30
duration = 7

X_train = np.array([df_train_normalized[i : i + history_points] for i in range(len(df_train_normalized) - history_points)])
y_train = np.array([df_train_normalized[i + history_points + future: i + history_points + future + duration, 0].sum() for i in range(len(df_train_normalized) - history_points - future)])
X_train = X_train[:len(y_train)]

X_test = np.array([df_test_normalized[i : i + history_points] for i in range(len(df_test_normalized) - history_points)])
y_test = np.array([df_test_normalized[i + history_points + future: i + history_points + future + duration, 0].sum() for i in range(len(df_test_normalized) - history_points - future)])
X_test = X_test[:len(y_test)]

In [ ]:
def lstm_model():
    tf.random.set_seed(20)
    np.random.seed(10)
    lstm_input = Input(shape=(history_points, 2), name='lstm_input')

    inputs = LSTM(30, name='first_layer')(lstm_input)
    inputs = Dense(10)(inputs)
    inputs = Dense(1)(inputs)
    output = Activation('linear', name='output')(inputs)

    model = Model(inputs=lstm_input, outputs=output)
    
    return model

model = lstm_model()

adam = optimizers.Adam(lr = 0.001)
model.compile(optimizer=adam, loss='mse')
model.fit(x=X_train, y=y_train, batch_size=4, epochs=5, shuffle=True, validation_data=(X_test, y_test))

In [ ]:
y_pred = model.predict(X_test)

fig, ax = plt.subplots(figsize=(8,6))

ax.plot(y_pred, label='LSTM prediction')
ax.plot(y_test, label='Ground_truth')

ax.set_title("Validation set")

ax.legend()

In [ ]:
y_pred = model.predict(X_train)

fig, ax = plt.subplots(figsize=(16,12))

ax.plot(y_pred, label='LSTM prediction')
ax.plot(y_train, label='Ground_truth')

ax.set_title("Training set")

ax.legend()

In [ ]:
def lstm_model():
    tf.random.set_seed(20)
    np.random.seed(10)
    lstm_input = Input(shape=(history_points, 2), name='lstm_input')

    inputs = LSTM(64, name='first_lstm_layer', return_sequences=True)(lstm_input)
    inputs = LSTM(32, name='second_lstm_layer')(inputs)
    inputs = Dense(10)(inputs)
    inputs = Dense(1)(inputs)
    output = Activation('linear', name='output')(inputs)

    model = Model(inputs=lstm_input, outputs=output)
    
    return model

model = lstm_model()

adam = optimizers.Adam(lr = 0.001)
model.compile(optimizer=adam, loss='mse')
model.fit(x=X_train, y=y_train, batch_size=4, epochs=5, shuffle=True, validation_data=(X_test, y_test))

In [ ]:
y_pred = model.predict(X_test)

fig, ax = plt.subplots(figsize=(8,6))

ax.plot(y_pred, label='LSTM prediction')
ax.plot(y_test, label='Ground_truth')

ax.set_title("Validation set")

ax.legend()

In [ ]:
y_pred = model.predict(X_train)

fig, ax = plt.subplots(figsize=(16,12))

ax.plot(y_pred, label='LSTM prediction')
ax.plot(y_train, label='Ground_truth')

ax.set_title("Training set")

ax.legend()

In [ ]:
df_A.index[0].weekday()

In [ ]:
df_A['weekday'] = df_A.index.weekday()

In [ ]:
 df_A.index.dt.weekday()

In [ ]:
weekdays = [idx.weekday() for idx in df_A.index]

In [ ]:
df_A['weekday'] = weekdays

In [ ]:
df_A

In [ ]:
df_A[df_A['weekday']==0]['A'].hist()

In [ ]:
df_A[df_A['weekday']==1]['A'].hist()

In [ ]:
df_A[df_A['weekday']==2]['A'].hist()

In [ ]:
df_A[df_A['weekday']==3]['A'].hist()

In [ ]:
df_A[df_A['weekday']==4]['A'].hist()

In [ ]:
df_A[df_A['weekday']==5]['A'].hist()

In [ ]:
df_A[df_A['weekday']==6]['A'].hist()